# Times obtained during auto analysis test (5 Orgs - 1 Peer Node per Org)

### Imports

In [1]:
import json
import time
import hashlib
import os
import requests
import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

### Global variables

In [2]:
API_ENDPOINT = "http://35.241.200.124:3000/api/"
NS = "ertis.uma.nuclear"

### Main function

In [3]:
def addMultipleAutomaticAnalysis():
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred)

    time_list = []
    for i in range(1, 101):
        filename = "acq"
        if i < 10:
            filename = filename + "00" + str(i) + ".txt"
        elif i < 100:
            filename = filename + "0" + str(i) + ".txt"
        else:
            filename = filename + str(i) + ".txt"

        #Download file from repository
        downloadFileFromRepository(filename)

        #Get file content
        acqData = getFileContent(filename)

        #Delete local file
        deleteLocalFile(filename)

        #Send transaction
        elapsed_time = addAutomaticAnalysis(i, acqData)
        time_list.append(elapsed_time)
        print(f"Elapsed time adding automatic analysis: {elapsed_time}\r\n")
        time.sleep(2)
    
    print(f"Fastest --> {min(time_list)}")
    print(f"Slowest --> {max(time_list)}")
    print(f"Average --> {sum(time_list)/len(time_list)}")

### Auxiliar functions

In [4]:
def downloadFileFromRepository(filename):
    bucket = storage.bucket("hyperledger-jte.appspot.com")

    blob = bucket.get_blob(filename)
    blob.download_to_filename(filename)

def checkHashSHA256(filename, hashStored):
    hashValue = sha256(filename)
    return hashValue == hashStored

def sha256(fname):
    hash_sha256 = hashlib.sha256()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_sha256.update(chunk)
    return hash_sha256.hexdigest()

def getFileContent(filename):
    content = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            content.append(int(lines[i]))
    return content

def deleteLocalFile(filename):
    if(os.path.exists(filename)):
        os.remove(filename)
    else:
        print(f"File {filename} does not exist")

def addAutomaticAnalysis(acqId, acqData, DEBUG=False):
    resource_url = f"{API_ENDPOINT}{NS}.AddAutomaticAnalysis"
    data = {
        "analysisId": acqId+200,
        "analysisDate": generateDateTime(),
        "acqId": acqId,
        "acqData": acqData
    }
    start_time = time.time()
    r = requests.post(resource_url, data=data)
    elapsed_time = time.time() - start_time
    if DEBUG:
        print(f"Elapsed time: {elapsed_time}")
        print(f"Response status code: {r.status_code}")
        print(r.json())
    return elapsed_time

def generateDateTime():
    x = str(datetime.datetime.now()).replace(" ", "T")
    x2 = x[:len(x)-3]+"Z"

    return x2

### Execution of the test

In [5]:
addMultipleAutomaticAnalysis()

Elapsed time adding automatic analysis: 19.43511176109314

Elapsed time adding automatic analysis: 19.068090677261353

Elapsed time adding automatic analysis: 24.165382146835327

Elapsed time adding automatic analysis: 19.16209602355957

Elapsed time adding automatic analysis: 19.07209086418152

Elapsed time adding automatic analysis: 23.92836880683899

Elapsed time adding automatic analysis: 19.27210235595703

Elapsed time adding automatic analysis: 18.667067527770996

Elapsed time adding automatic analysis: 19.341106176376343



ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))